In [ ]:
import ipyparallel as ipp

In [ ]:
ne = 640
p = 2
nps = [1, 2, 4, 8, 16]

labels = [
              'Poisson Mesh', 'Poisson Functions',
              'Poisson Dirichlet BCs', 'Poisson Neumann BCs',
              'Poisson Forms', 'Poisson Assemble',
              'Poisson Solve'
         ]

maxtimes = []
for np in nps:
    cluster = ipp.Cluster(engine_launcher_class="mpi", n=np)
    rc = cluster.start_and_connect_sync()
    view = rc[:]

    @view.remote(block=True)
    def poisson_parallel(ne, p, labels):
        from poisson_2d import solve_poisson_2d
        import dolfinx as df
        from mpi4py import MPI
        
        T_i = solve_poisson_2d(ne, p)
        
        print(T_i.function_space.mesh.comm.rank, T_i.function_space.mesh.comm.size)
        
        times = [df.common.timing(l)[1] for l in labels]
        maxtimes = T_i.function_space.mesh.comm.reduce(times, op=MPI.MAX)
        
        return maxtimes

    maxtimes.append(poisson_parallel(ne, p, labels)[0])


In [ ]:
print('\t'.join(['\t']+[repr(np) for np in nps]))
for l, label in enumerate(labels):
    print('\t'.join([label]+[repr(t[l]) for t in maxtimes]))

In [ ]:
cluster